In [ ]:
import argparse
import os
import shutil
import hashlib
import time
import logging
from pathlib import Path

def calculate_md5(filename, block_size=256*128):
    """Calculate MD5 of a file."""
    md5 = hashlib.md5()
    with open(filename, 'rb') as f:
        for chunk in iter(lambda: f.read(block_size), b''):
            md5.update(chunk)
    return md5.hexdigest()

def sync_folders(source, replica):
    """Synchronize two folders."""
    for src_dir, dirs, files in os.walk(source):
        dst_dir = src_dir.replace(source, replica, 1)
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)
            logging.info(f"Directory created: {dst_dir}")
        for file_ in files:
            src_file = os.path.join(src_dir, file_)
            dst_file = os.path.join(dst_dir, file_)
            if not os.path.exists(dst_file) or calculate_md5(src_file) != calculate_md5(dst_file):
                shutil.copy2(src_file, dst_file)
                logging.info(f"File copied/updated: {dst_file}")
        for file_ in os.listdir(dst_dir):
            src_file = os.path.join(src_dir, file_)
            dst_file = os.path.join(dst_dir, file_)
            if not os.path.exists(src_file):
                os.remove(dst_file)
                logging.info(f"File deleted: {dst_file}")
                
def setup_logging(log_file_path):
    """Setup logging to file and console."""
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s - %(levelname)s - %(message)s',
                        handlers=[logging.FileHandler(log_file_path),
                                  logging.StreamHandler()])

def main():
    parser = argparse.ArgumentParser(description="Synchronize two folders.")
    parser.add_argument('--source', required=True, help='C:\\Users\\Lospsy\\Desktop\\New folder\\Assigment\\SourceFolder')
    parser.add_argument('--replica', required=True, help='C:\\Users\\Lospsy\\Desktop\\New folder\\Assigment\\ReplicaFolder')
    parser.add_argument('--interval', type=int, default=60, help="Synchronization interval in seconds")
    parser.add_argument('--log', required=True, help='C:\\Users\\Lospsy\\Desktop\\New folder\\Assigment\\logfile.log')
    
    args = parser.parse_args()
    
    setup_logging(args.log)
    
    while True:
        logging.info("Starting synchronization")
        sync_folders(args.source, args.replica)
        logging.info("Synchronization completed")
        time.sleep(args.interval)

if __name__ == "__main__":
    main()
